In [4]:
import requests
from bs4 import BeautifulSoup
import googlemaps
import pandas as pd
from tqdm import tqdm   #for文進捗の可視化
import time

GOOGLEAPIKEY = ""
gmaps = googlemaps.Client(key=GOOGLEAPIKEY)

N=4200  #検索id数、4200が妥当
k=0

df_info=pd.DataFrame()
df_loc=pd.DataFrame()
df_NPC=pd.DataFrame()

for ParkingId in tqdm(range(N)):

    request_url = "https://npc-npc.co.jp/parking/prefecture-0/city-0/parking-" + str(ParkingId) + "/"

    res = requests.get(request_url)
    res.encoding = 'utf-8'

    # 初期化
    Name = ""
    Address = ""
    Charge = ""
    Status = ""
    ParkingSpaces = ""
    Restriction = ""
    Hours = ""
    Latitude = ""
    Longitude = ""

    if res.status_code==200:
        soup = BeautifulSoup(res.text, 'html.parser')

        ##############駐車場ID###############
        # print(ParkingId)

        ##############駐車場名###############
        Name = soup.select('div.clearfix h2')[0].text
        # print(Name)

        #駐車場情報テーブルの行名、行数を取得
        RowName = soup.select('table.tableSt tbody tr th')
        Rowcount = str(soup.select('table.tableSt tbody')).count("<tr>")

        #行名が一致する場合にデータを取得
        for i in range(Rowcount):
            ##############住所###############
            if "住所" in RowName[i].text:
                Address = soup.select('table.tableSt tbody tr td')[i].text
                # print(Address)
            ##############空満情報###############
            elif "駐車場ステータス" in RowName[i].text:
                Status=soup.select('table.tableSt tbody tr td p')[i].select("img")[0]["alt"]  #空/満ステータス
                # print(Status)
            ##############駐車台数###############
            elif "収容台数" in RowName[i].text:
                ParkingSpaces = soup.select('table.tableSt tbody tr td')[i].text
                # print(ParkingSpaces)
            ##############駐車制限###############
            elif "駐車制限" in RowName[i].text:
                Restriction = soup.select('table.tableSt tbody tr td')[i].text
                # print(Restriction)
            ##############営業時間###############
            elif "営業時間" in RowName[i].text:
                Hours = soup.select('table.tableSt tbody tr td')[i].text
                # print(Hours)

        ##############料金###############
        ChargeList = soup.select('#price p.priceLine')
        Charge=""
        for element in ChargeList:
            Charge += element.text +"\n"
            # print(Charge)

        infolist={"ParkingId":ParkingId,"Name":Name,"Address":Address,"Charge":Charge,"Status":Status,"ParkingSpaces":ParkingSpaces,"Restriction":Restriction,"Hours":Hours}
        df=pd.DataFrame(infolist,index=[k])
        df_info=pd.concat([df_info,df])
        k+=1

    time.sleep(0.5)

for j in tqdm(range(len(df_info))):
#GoogleAPIで住所から緯度/経度を取得
    result = gmaps.geocode(df_info.iloc[j]["Address"])

    ##############緯度###############
    Latitude = result[0]["geometry"]["location"]["lat"]
    # print (Latitude)

    ##############経度###############
    Longitude = result[0]["geometry"]["location"]["lng"]
    # print (Longitude)

    list2={"ParkingId":df_info.iloc[j]["ParkingId"],"Latitude":Latitude,"Longitude":Longitude}
    df2=pd.DataFrame(list2,index=[j])
    df_loc=pd.concat([df_loc,df2])

df_NPC=pd.merge(df_info,df_loc,on="ParkingId")

100%|██████████| 35/35 [00:02<00:00, 15.40it/s]
